# test

In [19]:
pip install pytest


In [27]:
import cybermicrogrid

In [26]:
!git clone https://github.com/MohamedMassaoudi-code/cybermicrogrid.git
%cd cybermicrogrid
!pip install -e .


Cloning into 'cybermicrogrid'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 94 (delta 47), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (94/94), 50.49 KiB | 2.97 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/content/cybermicrogrid/pydnp3/dnp3/build/cybermicrogrid/cybermicrogrid
Obtaining file:///content/cybermicrogrid/pydnp3/dnp3/build/cybermicrogrid/cybermicrogrid
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cybermicrogrid
    Found existing installation: cybermicrogrid 0.1.0
    Uninstalling cybermicrogrid-0.1.0:
      Successfully uninstalled cybermicrogrid-0.1.0
  Running setup.py develop for cybermicrogrid


In [28]:
# tests/test_library.py

import pytest
from torch_geometric.data import Data

# Import the relevant modules from your library
from cybermicrogrid.networks import create_campus_microgrid_network
from cybermicrogrid.utils import ensure_features
from cybermicrogrid.environment import PowerGridEnvironment
from cybermicrogrid.agents import DummyAgent
from cybermicrogrid.training import train_value_based_agent


In [31]:
import numpy as np
import pandas as pd
from cybermicrogrid.networks import create_campus_microgrid_network
from cybermicrogrid.utils import ensure_features
from cybermicrogrid.agents import DQNAgent
from cybermicrogrid.training import train_value_based_agent
from torch_geometric.data import Data

# List all supported attack modes
attack_modes = [
    "NONE", "FDI", "MITM", "DOS", "REPLAY", "DDOS", "APT", "RANSOMWARE", "PHISHING",
    "ZERO_DAY", "INSIDER", "GPS_SPOOF", "SUPPLY_CHAIN", "CONTROL_HIJACK", "CASCADING",
    "SENSOR_JAM", "DATA_EXFIL", "MALWARE", "LOAD_ALTERING", "ENERGY_STORAGE_MANIPULATION", "DG_TAMPERING",
    "SYNCHRONIZATION", "MARKET_MANIPULATION", "FREQUENCY_CONTROL", "VOLTAGE_STABILITY",
    "BLACK_START", "COORDINATION"
    # Optionally add "DNP3_ATTACK" if using the DNP3-enabled environment
]

# Create the microgrid network and initial state data
net = create_campus_microgrid_network()
nbuses = net.bus.shape[0]
data = Data(num_nodes=nbuses)
data = ensure_features(data)

results = {}

# Loop through each attack mode and train a DQN agent
for mode in attack_modes:
    print(f"--- Training for attack mode: {mode} ---")

    # Use zero attack strength for "NONE" mode
    strength = 0.0 if mode.upper() == "NONE" else 0.1

    # Initialize a DQNAgent: state dimension = nbuses, action dimension = number of lines
    agent = DQNAgent(state_dim=nbuses, action_dim=len(net.line))

    # Train the agent for a small number of episodes (adjust num_episodes as needed)
    env, rewards, freq, power, steps, times = train_value_based_agent(
        pp_net=net,
        data=data,
        agent=agent,
        num_episodes=10,
        attack_mode=mode,
        attack_strength=strength
    )

    # Compute average performance metrics
    avg_reward = np.mean(rewards) if rewards else None
    avg_freq = np.mean(freq) if freq else None
    avg_power = np.mean(power) if power else None

    results[mode.upper()] = {
        "Average Reward": avg_reward,
        "Average Frequency (Hz)": avg_freq,
        "Average Active Power (MW)": avg_power,
    }
    print(f"Mode: {mode.upper()}, Avg Reward: {avg_reward:.2f}, Avg Freq: {avg_freq:.2f} Hz, Avg Power: {avg_power:.2f} MW")
    print()

# Create a summary DataFrame and print it
results_df = pd.DataFrame.from_dict(results, orient="index")
results_df.index.name = "Attack Mode"
print("\n=== DQN Performance Summary ===")
print(results_df.to_string())


--- Training for attack mode: NONE ---
[NONE][Episode 001/10] Reward: -1850.15, Steps: 20, Time: 0.244s
[NONE][Episode 002/10] Reward: -3942.34, Steps: 20, Time: 0.273s
[NONE][Episode 003/10] Reward: -3719.71, Steps: 20, Time: 0.295s
[NONE][Episode 004/10] Reward: -3416.83, Steps: 20, Time: 0.287s
[NONE][Episode 005/10] Reward: -4427.82, Steps: 20, Time: 0.302s
[NONE][Episode 006/10] Reward: -2880.15, Steps: 20, Time: 0.278s
[NONE][Episode 007/10] Reward: -1771.11, Steps: 20, Time: 0.266s
[NONE][Episode 008/10] Reward: -2255.34, Steps: 20, Time: 0.287s
[NONE][Episode 009/10] Reward: -3148.82, Steps: 20, Time: 0.275s
[NONE][Episode 010/10] Reward: -3627.31, Steps: 20, Time: 0.425s
Mode: NONE, Avg Reward: -3103.96, Avg Freq: 45.07 Hz, Avg Power: 1.79 MW

--- Training for attack mode: FDI ---
[FDI][Episode 001/10] Reward: -4974.77, Steps: 20, Time: 0.398s
[FDI][Episode 002/10] Reward: -1779.89, Steps: 20, Time: 0.370s
[FDI][Episode 003/10] Reward: -2378.37, Steps: 20, Time: 0.403s
[FDI][E